In [ ]:

# # Import required libraries
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, accuracy_score

# # Load the training dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")

# # Check for missing values in the "Class" column
# print("Number of missing values in 'Class' before cleaning:", train_data["Class"].isna().sum())

# # Drop rows with missing values in the "Class" column
# train_data = train_data.dropna(subset=["Class"])

# # Map non-numeric labels to integers
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure there are no NaN values after mapping
# train_data = train_data.dropna(subset=["Class"])
# print("Number of missing values in 'Class' after mapping and cleaning:", train_data["Class"].isna().sum())

# # Split the training data into train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
# )

# # Ensure there are no NaN values in training and validation labels
# print("NaN in y_train:", pd.isna(y_train).sum())
# print("NaN in y_val:", pd.isna(y_val).sum())

# # Text Vectorization using TF-IDF
# vectorizer = TfidfVectorizer(max_features=5000)
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_val_tfidf = vectorizer.transform(X_val)

# # Initialize and Train the Logistic Regression Model
# model = LogisticRegression()
# model.fit(X_train_tfidf, y_train)

# # Evaluate on the validation set
# y_val_pred = model.predict(X_val_tfidf)
# print("Validation Classification Report:\n", classification_report(y_val, y_val_pred))
# print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))

# # Load the test dataset
# test_data = pd.read_csv("/kaggle/input/test-data/test_predictions.csv")

# # Vectorize the test data
# X_test_tfidf = vectorizer.transform(test_data["Text"])

# # Make predictions on the test dataset
# test_predictions = model.predict(X_test_tfidf)

# # Convert predictions back to original label names
# reverse_label_mapping = {v: k for k, v in label_mapping.items()}
# test_predictions_labels = [reverse_label_mapping[pred] for pred in test_predictions]

# # Save predictions to a CSV file
# output = pd.DataFrame({"Text": test_data["Text"], "Predicted_Class": test_predictions_labels})
# output.to_csv("test_predictions.csv", index=False)
# print("Predictions saved to 'test_predictions.csv'.")


In [ ]:
pip install transformers datasets


In [ ]:
!pip install evaluate


In [ ]:
# # Import required libraries
# import pandas as pd
# from datasets import Dataset
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score

# # Load the dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
# test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# # Map the labels
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure no missing values
# train_data = train_data.dropna(subset=["Class"])

# # Split the dataset into training and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.1, random_state=42
# )

# # Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Tokenize the dataset
# def preprocess_function(texts, labels):
#     tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128)
#     tokenized["labels"] = labels
#     return tokenized

# train_encodings = preprocess_function(train_texts, list(train_labels))
# val_encodings = preprocess_function(val_texts, list(val_labels))

# # Convert tokenized data to PyTorch datasets
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __len__(self):
#         return len(self.encodings["input_ids"])

#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# train_dataset = CustomDataset(train_encodings)
# val_dataset = CustomDataset(val_encodings)

# # Initialize the model
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)

# # Define the compute_metrics function
# def compute_metrics(pred):
#     predictions, labels = pred
#     preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
#     labels = torch.tensor(labels).int()
    
#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds)
#     recall = recall_score(labels, preds)
    
#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#     }

# # Define the Trainer arguments
# training_args = TrainingArguments(
#     output_dir="./results",               # Output directory
#     eval_strategy="epoch",               # Evaluate every epoch
#     save_strategy="epoch",               # Save every epoch
#     learning_rate=5e-5,                  # Learning rate
#     per_device_train_batch_size=16,      # Batch size for training
#     per_device_eval_batch_size=16,       # Batch size for evaluation
#     num_train_epochs=3,                  # Number of epochs
#     weight_decay=0.01,                   # Weight decay
#     logging_dir="./logs",                # Logging directory
#     logging_steps=10,                    # Log every 10 steps
#     report_to="none",                    # Disable W&B logging
# )

# # Define the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,  # Add validation dataset
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,  # Add the metrics function
# )

# # Train the model
# trainer.train()

# # Make predictions on the test set
# test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
# test_dataset = CustomDataset(test_encodings)

# predictions = trainer.predict(test_dataset)
# preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
# pred_labels = (preds > 0.5).astype(int).flatten()

# # Save predictions
# test_data["Predicted_Class"] = pred_labels
# test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
# test_data.to_csv("test_predictions.csv", index=False)
# print("Predictions saved to 'test_predictions.csv'.")


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# Map the labels
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)

# Ensure no missing values
train_data = train_data.dropna(subset=["Class"])

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
)

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

# Tokenize the dataset
def preprocess_function(texts, labels):
    tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = labels
    return tokenized

train_encodings = preprocess_function(train_texts, list(train_labels))
val_encodings = preprocess_function(val_texts, list(val_labels))

# Convert tokenized data to PyTorch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

# Initialize the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=1)

# Define the compute_metrics function
def compute_metrics(pred):
    predictions, labels = pred
    preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
    labels = torch.tensor(labels).int()
    
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0  # F1 Score
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,  # F1 Score
    }

# Define the Trainer arguments
training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    eval_strategy="epoch",                # Evaluate every epoch
    save_strategy="epoch",                # Save every epoch
    learning_rate=5e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Batch size for training
    per_device_eval_batch_size=16,        # Batch size for evaluation
    num_train_epochs=7,                   # Number of epochs
    weight_decay=0.003,                    # Weight decay
    logging_dir="./logs",                 # Logging directory
    logging_steps=10,                     # Log every 10 steps
    report_to="none",                     # Disable W&B logging
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Add validation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add the metrics function
)

# Train the model
trainer.train()

# Make predictions on the test set
test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
test_dataset = CustomDataset(test_encodings)

predictions = trainer.predict(test_dataset)
preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
pred_labels = (preds > 0.5).astype(int).flatten()

# Save predictions
test_data["Predicted_Class"] = pred_labels
test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
test_data.to_csv("test_predictions_distilbert.csv", index=False)
print("Predictions saved to 'test_predictions_distilbert.csv'.") 


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# Map the labels
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)

# Ensure no missing values
train_data = train_data.dropna(subset=["Class"])

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.1, random_state=42
)

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Tokenize the dataset
def preprocess_function(texts, labels):
    tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    tokenized["labels"] = labels
    return tokenized

train_encodings = preprocess_function(train_texts, list(train_labels))
val_encodings = preprocess_function(val_texts, list(val_labels))

# Convert tokenized data to PyTorch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

# Initialize the model
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)

# Define the compute_metrics function
def compute_metrics(pred):
    predictions, labels = pred
    preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
    labels = torch.tensor(labels).int()
    
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Define the TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    eval_strategy="epoch",                # Evaluate every epoch
    save_strategy="epoch",                # Save every epoch
    learning_rate=5e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Batch size for training
    per_device_eval_batch_size=16,        # Batch size for evaluation
    num_train_epochs=7,                   # Number of epochs
    weight_decay=0.003,                   # Weight decay
    logging_dir="./logs",                 # Logging directory
    logging_steps=10,                     # Log every 10 steps
    report_to="none",                     # Disable W&B logging
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Make predictions on the test set
test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
test_dataset = CustomDataset(test_encodings)

predictions = trainer.predict(test_dataset)
preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
pred_labels = (preds > 0.5).astype(int).flatten()

# Save predictions with only "Id" and "Labels"
test_data["Predicted_Class"] = pred_labels
test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
test_data.to_csv("test_predictions_bert.csv", index=False)
print("Predictions saved to 'test_predictions_bert.csv'.") 

In [ ]:
# import pandas as pd

# # Load the predictions
# test_predictions = pd.read_csv("test_predictions_bert.csv")

# # Create the file with only 'id' and 'Predicted_Class'
# prediction_file = test_predictions[['id', 'Predicted_Class']]

# # Save the file
# prediction_file.to_csv("test_predictions_bert_only_id_labels.csv", index=False)

# print("Prediction file with 'id' and 'Predicted_Class' saved as 'test_predictions_bert_only_id_labels.csv'.")


In [ ]:
# import pandas as pd
# from datasets import Dataset
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score

# # Load the dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
# test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# # Map the labels
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure no missing values
# train_data = train_data.dropna(subset=["Class"])

# # Split the dataset into training and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.1, random_state=42
# )

# # Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Tokenize the dataset
# def preprocess_function(texts, labels):
#     tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
#     tokenized["labels"] = labels
#     return tokenized

# train_encodings = preprocess_function(train_texts, list(train_labels))
# val_encodings = preprocess_function(val_texts, list(val_labels))

# # Convert tokenized data to PyTorch datasets
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __len__(self):
#         return len(self.encodings["input_ids"])

#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# train_dataset = CustomDataset(train_encodings)
# val_dataset = CustomDataset(val_encodings)

# # Initialize the model
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)

# # Define the compute_metrics function
# def compute_metrics(pred):
#     predictions, labels = pred
#     preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
#     labels = torch.tensor(labels).int()
    
#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds)
#     recall = recall_score(labels, preds)
#     f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0  # F1 Score
    
#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#         "f1": f1,  # F1 Score
#     }

# # Define the TrainingArguments
# training_args = TrainingArguments(
#     output_dir="./results",               # Output directory
#     eval_strategy="epoch",                # Evaluate every epoch
#     save_strategy="epoch",                # Save every epoch
#     learning_rate=5e-5,                   # Learning rate (try adjusting this)
#     per_device_train_batch_size=16,       # Batch size for training
#     per_device_eval_batch_size=16,        # Batch size for evaluation
#     num_train_epochs=7,                   # Number of epochs
#     weight_decay=0.003,                    # Weight decay
#     logging_dir="./logs",                 # Logging directory
#     logging_steps=10,                     # Log every 10 steps
#     report_to="none",                     # Disable W&B logging
# )

# # Define the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,  # Add validation dataset
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,  # Add the metrics function
# )

# # Train the model
# trainer.train()

# # Make predictions on the test set
# test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
# test_dataset = CustomDataset(test_encodings)

# predictions = trainer.predict(test_dataset)
# preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
# pred_labels = (preds > 0.5).astype(int).flatten()

# # Save predictions
# test_data["Labels"] = pred_labels
# test_data["Labels"] = test_data["Labels"].map({0: "Non-Abusive", 1: "Abusive"})
# #test_data.to_csv("test_predictions_bert.csv", index=False)
# #print("Predictions saved to 'test_predictions_bert.csv'.") 
# # Load the predictions
# #test_predictions = pd.read_csv("test_predictions_bert.csv")

# # Create the file with only 'id' and 'Predicted_Class'
# prediction_file = test_data[['id', 'Labels']]

# # Save the file
# prediction_file.to_csv("test_predictions_bert_only_id_labels1.csv", index=False)

# print("Prediction file with 'id' and 'Predicted_Class' saved as 'test_predictions_bert_only_id_labels.csv'.")



In [ ]:
# import pandas as pd

# # Load the predictions
# test_predictions = pd.read_csv("test_predictions_bert.csv")

# # Create the file with only 'id' and 'Predicted_Class'
# prediction_file = test_predictions[['id', 'Predicted_Class']]

# # Save the file
# prediction_file.to_csv("test_predictions_bert_only_id_labels.csv", index=False)

# print("Prediction file with 'id' and 'Predicted_Class' saved as 'test_predictions_bert_only_id_labels.csv'.")


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# Map the labels
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)

# Ensure no missing values
train_data = train_data.dropna(subset=["Class"])

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.1, random_state=42
)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the dataset
def preprocess_function(texts, labels):
    tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = labels
    return tokenized

train_encodings = preprocess_function(train_texts, list(train_labels))
val_encodings = preprocess_function(val_texts, list(val_labels))

# Convert tokenized data to PyTorch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

# Initialize the model
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

# Define the compute_metrics function
def compute_metrics(pred):
    predictions, labels = pred
    preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
    labels = torch.tensor(labels).int()
    
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0  # F1 Score
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,  # F1 Score
    }

# Define the Trainer arguments
training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    eval_strategy="epoch",                # Evaluate every epoch
    save_strategy="epoch",                # Save every epoch
    learning_rate=5e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Batch size for training
    per_device_eval_batch_size=16,        # Batch size for evaluation
    num_train_epochs=3,                   # Number of epochs
    weight_decay=0.003,                    # Weight decay
    logging_dir="./logs",                 # Logging directory
    logging_steps=10,                     # Log every 10 steps
    report_to="none",                     # Disable W&B logging
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Add validation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add the metrics function
)

# Train the model
trainer.train()

# Make predictions on the test set
test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
test_dataset = CustomDataset(test_encodings)

predictions = trainer.predict(test_dataset)
preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
pred_labels = (preds > 0.5).astype(int).flatten()

# Save predictions
test_data["Predicted_Class"] = pred_labels
test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
test_data.to_csv("test_predictions_xlmr.csv", index=False)
print("Predictions saved to 'test_predictions_xlmr.csv'.")


In [ ]:
# # Import required libraries
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import classification_report, accuracy_score

# # Load the training dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")

# # Check for missing values in the "Class" column
# print("Number of missing values in 'Class' before cleaning:", train_data["Class"].isna().sum())

# # Drop rows with missing values in the "Class" column
# train_data = train_data.dropna(subset=["Class"])

# # Map non-numeric labels to integers
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure there are no NaN values after mapping
# train_data = train_data.dropna(subset=["Class"])
# print("Number of missing values in 'Class' after mapping and cleaning:", train_data["Class"].isna().sum())

# # Split the training data into train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
# )

# # Ensure there are no NaN values in training and validation labels
# print("NaN in y_train:", pd.isna(y_train).sum())
# print("NaN in y_val:", pd.isna(y_val).sum())

# # Define a function to train and evaluate models
# def train_and_evaluate(models, feature_extractor, feature_name):
#     # Transform text data using the feature extractor
#     X_train_features = feature_extractor.fit_transform(X_train)
#     X_val_features = feature_extractor.transform(X_val)
    
#     # Iterate over the models and train them
#     for model_name, model in models.items():
#         print(f"\nTraining {model_name} with {feature_name} features...")
#         model.fit(X_train_features, y_train)
#         y_val_pred = model.predict(X_val_features)
        
#         # Evaluate the model
#         print(f"\nValidation Results for {model_name} ({feature_name}):")
#         print("Classification Report:\n", classification_report(y_val, y_val_pred))
#         print("Accuracy:", accuracy_score(y_val, y_val_pred))

# # Define models to train
# models = {
#     "Logistic Regression": LogisticRegression(),
#     "SVM": SVC(),
#     "Random Forest": RandomForestClassifier(),
#     "Multinomial Naive Bayes": MultinomialNB()
# }

# # Evaluate with TF-IDF features
# print("\nUsing TF-IDF features:")
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)
# train_and_evaluate(models, tfidf_vectorizer, "TF-IDF")

# # Evaluate with Bag of Words (BoW) features
# print("\nUsing Bag of Words (BoW) features:")
# bow_vectorizer = CountVectorizer(max_features=5000)
# train_and_evaluate(models, bow_vectorizer, "BoW")

# # Load the test dataset
# test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# # Transform test data using the TF-IDF vectorizer
# X_test_tfidf = tfidf_vectorizer.transform(test_data["Text"])

# # Make predictions on the test dataset using the best-performing model (e.g., Logistic Regression)
# best_model = LogisticRegression()  # Replace with your chosen model if needed
# best_model.fit(tfidf_vectorizer.fit_transform(X_train), y_train)  # Retrain on the full training data
# test_predictions = best_model.predict(X_test_tfidf)

# # Convert predictions back to original label names
# reverse_label_mapping = {v: k for k, v in label_mapping.items()}
# test_predictions_labels = [reverse_label_mapping[pred] for pred in test_predictions]

# # Save predictions to a CSV file
# output = pd.DataFrame({"Text": test_data["Text"], "Predicted_Class": test_predictions_labels})
# output.to_csv("test_predictions.csv", index=False)
# print("Predictions saved to 'test_predictions.csv'.")


In [ ]:
# Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Load the training dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")

# Check for missing values in the "Class" column
print("Number of missing values in 'Class' before cleaning:", train_data["Class"].isna().sum())

# Drop rows with missing values in the "Class" column
train_data = train_data.dropna(subset=["Class"])

# Map non-numeric labels to integers
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)

# Ensure there are no NaN values after mapping
train_data = train_data.dropna(subset=["Class"])
print("Number of missing values in 'Class' after mapping and cleaning:", train_data["Class"].isna().sum())

# Split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
)

# Ensure there are no NaN values in training and validation labels
print("NaN in y_train:", pd.isna(y_train).sum())
print("NaN in y_val:", pd.isna(y_val).sum())

# Define a function to train and evaluate models
def train_and_evaluate(models, feature_extractor, feature_name):
    # Transform text data using the feature extractor
    X_train_features = feature_extractor.fit_transform(X_train)
    X_val_features = feature_extractor.transform(X_val)
    
    # Iterate over the models and train them
    for model_name, model in models.items():
        print(f"\nTraining {model_name} with {feature_name} features...")
        model.fit(X_train_features, y_train)
        y_val_pred = model.predict(X_val_features)
        
        # Evaluate the model
        print(f"\nValidation Results for {model_name} ({feature_name}):")
        print("Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
        accuracy = accuracy_score(y_val, y_val_pred)
        print(f"Accuracy: {accuracy:.4f}")

# Define models to train
models = {
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "Multinomial Naive Bayes": MultinomialNB()
}

# Evaluate with TF-IDF features
print("\nUsing TF-IDF features:")
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
train_and_evaluate(models, tfidf_vectorizer, "TF-IDF")

# Evaluate with Bag of Words (BoW) features
print("\nUsing Bag of Words (BoW) features:")
bow_vectorizer = CountVectorizer(max_features=5000)
train_and_evaluate(models, bow_vectorizer, "BoW")

# Load the test dataset
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# Transform test data using the TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(test_data["Text"])

# Make predictions on the test dataset using the best-performing model (e.g., Logistic Regression)
best_model = LogisticRegression()  # Replace with your chosen model if needed
best_model.fit(tfidf_vectorizer.fit_transform(X_train), y_train)  # Retrain on the full training data
test_predictions = best_model.predict(X_test_tfidf)

# Convert predictions back to original label names
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
test_predictions_labels = [reverse_label_mapping[pred] for pred in test_predictions]

# Save predictions to a CSV file
output = pd.DataFrame({"Text": test_data["Text"], "Predicted_Class": test_predictions_labels})
output.to_csv("test_predictions.csv", index=False)
print("Predictions saved to 'test_predictions.csv'.")


In [ ]:
# import pandas as pd
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score

# # Load the dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
# test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# # Map the labels
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure no missing values
# train_data = train_data.dropna(subset=["Class"])

# # Split the dataset into training and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
# )

# # Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Tokenize the dataset
# def preprocess_function(texts, labels):
#     tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
#     tokenized["labels"] = labels
#     return tokenized

# train_encodings = preprocess_function(train_texts, list(train_labels))
# val_encodings = preprocess_function(val_texts, list(val_labels))

# # Convert tokenized data to PyTorch datasets
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __len__(self):
#         return len(self.encodings["input_ids"])

#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# train_dataset = CustomDataset(train_encodings)
# val_dataset = CustomDataset(val_encodings)

# # Initialize the model
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)

# # Define the compute_metrics function
# def compute_metrics(pred):
#     predictions, labels = pred
#     preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
#     labels = torch.tensor(labels).int()
    
#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds)
#     recall = recall_score(labels, preds)
#     f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#         "f1": f1,
#     }

# # Define the TrainingArguments
# training_args = TrainingArguments(
#     output_dir="./results",               # Output directory
#     eval_strategy="epoch",                # Evaluate every epoch
#     save_strategy="epoch",                # Save every epoch
#     learning_rate=5e-5,                   # Learning rate
#     per_device_train_batch_size=16,       # Batch size for training
#     per_device_eval_batch_size=16,        # Batch size for evaluation
#     num_train_epochs=7,                   # Number of epochs
#     weight_decay=0.003,                   # Weight decay
#     logging_dir="./logs",                 # Logging directory
#     logging_steps=10,                     # Log every 10 steps
#     report_to="none",                     # Disable W&B logging
# )

# # Define the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

# # Train the model
# trainer.train()

# # Make predictions on the test set
# test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
# test_dataset = CustomDataset(test_encodings)

# predictions = trainer.predict(test_dataset)
# preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
# pred_labels = (preds > 0.5).astype(int).flatten()

# # Save predictions with only "Id" and "Labels"
# test_data["Predicted_Class"] = pred_labels
# test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
# test_data.to_csv("test_predictions_bert.csv", index=False)
# print("Predictions saved to 'test_predictions_bert.csv'.") 

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import FastText
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the training dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")

# Preprocess the data
print("Number of missing values in 'Class' before cleaning:", train_data["Class"].isna().sum())
train_data = train_data.dropna(subset=["Class"])
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)
train_data = train_data.dropna(subset=["Class"])
print("Number of missing values in 'Class' after mapping and cleaning:", train_data["Class"].isna().sum())

# Split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
)

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_train_padded = pad_sequences(X_train_seq, padding="post")
X_val_padded = pad_sequences(X_val_seq, padding="post", maxlen=X_train_padded.shape[1])

# Load FastText embeddings
def load_embeddings(embedding_dim=100):
    sentences = [text.split() for text in X_train]
    model = FastText(sentences, vector_size=embedding_dim, window=5, min_count=1, sg=1)
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
    for word, i in tokenizer.word_index.items():
        if word in model.wv:
            embedding_matrix[i] = model.wv[word]
    return embedding_matrix

embedding_matrix = load_embeddings(embedding_dim=100)

# Define CNN model
def cnn_model(input_length, embedding_matrix):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0], 
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix], 
                  input_length=input_length, 
                  trainable=False),
        Conv1D(filters=128, kernel_size=5, activation="relu"),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Define BiLSTM model
def bilstm_model(input_length, embedding_matrix):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0], 
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix], 
                  input_length=input_length, 
                  trainable=False),
        Bidirectional(LSTM(128, return_sequences=True)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Define CNN+BiLSTM model
def cnn_bilstm_model(input_length, embedding_matrix):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0], 
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix], 
                  input_length=input_length, 
                  trainable=False),
        Conv1D(filters=128, kernel_size=5, activation="relu"),
        MaxPooling1D(pool_size=2),
        Bidirectional(LSTM(128, return_sequences=True)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Train and evaluate models
def train_and_evaluate_dl_model(model, model_name):
    print(f"\nTraining {model_name}...")
    model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_data=(X_val_padded, y_val))
    y_val_pred = (model.predict(X_val_padded) > 0.5).astype("int32").flatten()
    print(f"\nValidation Results for {model_name}:")
    print("Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
    accuracy = accuracy_score(y_val, y_val_pred)
    print(f"Accuracy: {accuracy:.4f}")

# Train CNN
cnn = cnn_model(X_train_padded.shape[1], embedding_matrix)
train_and_evaluate_dl_model(cnn, "CNN")

# Train BiLSTM
bilstm = bilstm_model(X_train_padded.shape[1], embedding_matrix)
train_and_evaluate_dl_model(bilstm, "BiLSTM")

# Train CNN+BiLSTM
cnn_bilstm = cnn_bilstm_model(X_train_padded.shape[1], embedding_matrix)
train_and_evaluate_dl_model(cnn_bilstm, "CNN+BiLSTM")

# Load the test dataset
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")
X_test_seq = tokenizer.texts_to_sequences(test_data["Text"])
X_test_padded = pad_sequences(X_test_seq, padding="post", maxlen=X_train_padded.shape[1])

# Make predictions on test dataset using the best-performing model (e.g., CNN+BiLSTM)
best_model = cnn_bilstm  # Replace with your chosen model
test_predictions = (best_model.predict(X_test_padded) > 0.5).astype("int32").flatten()

# Convert predictions back to original label names
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
test_predictions_labels = [reverse_label_mapping[pred] for pred in test_predictions]

# Save predictions to a CSV file
output = pd.DataFrame({"Text": test_data["Text"], "Predicted_Class": test_predictions_labels})
output.to_csv("test_predictions.csv", index=False)
print("Predictions saved to 'test_predictions.csv'.")


Number of missing values in 'Class' before cleaning: 0
Number of missing values in 'Class' after mapping and cleaning: 0


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Training CNN...
Epoch 1/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.4997 - loss: 0.7009 - val_accuracy: 0.5466 - val_loss: 0.6895
Epoch 2/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5524 - loss: 0.6861 - val_accuracy: 0.5466 - val_loss: 0.6865
Epoch 3/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5815 - loss: 0.6710 - val_accuracy: 0.5341 - val_loss: 0.6858
Epoch 4/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5653 - loss: 0.6692 - val_accuracy: 0.5627 - val_loss: 0.6848
Epoch 5/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5858 - loss: 0.6512 - val_accuracy: 0.5681 - val_loss: 0.6835
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Validation Results for CNN:
Classification Report:
               precision    recall  f1-score   support

         0.0     0.5685    0.5907    0.5794       281
         1.0     0.5677    0.5451    0.5562       277

    accuracy                         0.5681       558
   macro avg     0.5681    0.5679    0.5678   

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.4832 - loss: 0.7245 - val_accuracy: 0.5072 - val_loss: 0.6926
Epoch 2/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5017 - loss: 0.7131 - val_accuracy: 0.5018 - val_loss: 0.6921
Epoch 3/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5151 - loss: 0.6951 - val_accuracy: 0.5179 - val_loss: 0.6915
Epoch 4/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5255 - loss: 0.6910 - val_accuracy: 0.5036 - val_loss: 0.6930
Epoch 5/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5280 - loss: 0.6954 - val_accuracy: 0.4964 - val_loss: 0.6926
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

Validation Results for BiLSTM:
Classification Report:
               precision    recall  f1-score   support

         0.0     0.0000    0.0000    0.0000       281
         1.0     0.4964    1.0000    0.6635       277

    accuracy                         0.4964       558
   macro avg     0.2482    0.5000    0.3317       558
weighted avg

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/em

70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.5238 - loss: 0.7119 - val_accuracy: 0.5072 - val_loss: 0.6930
Epoch 2/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4915 - loss: 0.6959 - val_accuracy: 0.5036 - val_loss: 0.6928
Epoch 3/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5088 - loss: 0.6930 - val_accuracy: 0.5036 - val_loss: 0.6947
Epoch 4/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5333 - loss: 0.6898 - val_accuracy: 0.5090 - val_loss: 0.7250
Epoch 5/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5280 - loss: 0.6982 - val_accuracy: 0.5681 - val_loss: 0.6884
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

Validation Results for CNN+BiLSTM:
Classification Report:
               precision    recall  f1-score   support

         0.0     0.5699    0.5801    0.5750       281
         1.0     0.5662    0.5560    0.5610       277

    accuracy                         0.5681       558
   macro avg     0.5681    0.5680    0.5680       558
weighted